generate conf
how to launch: python3 run_oblivious_c2s.py --conf computerouting2/cluster_c.conf --worker 1 --maxGB 5 --waitSec 30

In [3]:
trafficname = "cluster_c"
scaledown = 100000
with open(f"/home/annzhou/DRing/src/emp/datacentre/computerouting2/{trafficname}.conf",'w') as f:
    for H in range(1,25):
        f.write(f"python3 computerouting2/testtemporalH_meta.py {trafficname} {scaledown} {H}\n")

python3 run_oblivious_c2s.py --conf computerouting2/unv1.conf --worker 1 --maxGB 5 --waitSec 30

In [6]:
trafficname = "unv1"
scaledown = 10000000
with open(f"/home/annzhou/DRing/src/emp/datacentre/computerouting2/{trafficname}.conf",'w') as f:
    for H in range(1,5):
        f.write(f"python3 computerouting2/testtemporalH_wisc.py {trafficname} {scaledown} {H}\n")